In [1]:
#quality>5 = good
#quality<6 = bad
##Dataset from:
##P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis.
##Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.
##Author: Nishank Raisinghani
##Project: Using XGBoost and SciKit Learn to predict the quality of wine using a classifier


##Import
import numpy as np
import pandas as pd
from pandas import DataFrame
from sklearn import tree
from sklearn.preprocessing import scale
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.preprocessing import StandardScaler


##Load in data
df = pd.read_csv("winequality-white1.csv", sep=";")


##Make label data binary: turn 0-10 scale into good and bad
d = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:1, 7:1, 8:1, 9:1, 10:1}
df['quality'] = df['quality'].map(d)

##Define your features
features = list(df.columns[:11])

##split data up into training and testing
from sklearn.model_selection import train_test_split
traindf, testdf = train_test_split(df, test_size=0.2)

##define your features and label values
scaler = StandardScaler()
x = traindf[features]
scaler.fit(x)
x=pd.DataFrame(scaler.transform(x))
y = traindf['quality']
testx = testdf[features]
scaler.fit(testx)
testx=pd.DataFrame(scaler.transform(testx))
testy = testdf['quality']

##Turn into DMatrix; this is how XGBoost likes its data
training = xgb.DMatrix(x, label =y)
testing = xgb.DMatrix(testx, label=testy)

##Enter in your parameters; I am training 2 models, one is a softmax classification and one is a binary logistic classification
param = {
    'max_depth': 4,
    'eta': 0.3,
    'objective': 'multi:softmax',
    'num_class': 2} 
epochs = 10 

model = xgb.train(param, training, epochs)

param2 = {
    'max_depth': 4,
    'eta': 0.2,
    'objective': 'binary:logistic'} 
epochs = 8 

model2 = xgb.train(param2, training, epochs)

##Run your model on the testing data
predictions = model.predict(testing)
predictions2 = model2.predict(testing)
predictionDF = DataFrame({'predictions':predictions})
predictionDF2 = DataFrame({'predictions':predictions2})

##Since the binary logistic will give you a value between 0 and 1, and you only want a value of 0 or 1 you need to map this data to make it 0s and 1s
predictionDF2['predictions'] = predictionDF2['predictions'].map(lambda x: 0.0 if x <0.5 else 1.0)

##Use these 0 and 1 values to turn the predictions into 'good' and 'bad' so it looks presentable
d = {1:'good', 0:'bad'}
predictionDF['predictions']=predictionDF['predictions'].map(d)
predictionDF2['predictions']=predictionDF2['predictions'].map(d)
testdf['quality']=testdf['quality'].map(d)

##return the 
testdf['predictions2'] = predictionDF2['predictions'].values
testdf['predictions']=predictionDF['predictions'].values
testdf.head(50)



[10:44:48] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:44:48] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


<ipython-input-1-524c8d5248e4>:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testdf['quality']=testdf['quality'].map(d)
<ipython-input-1-524c8d5248e4>:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testdf['predictions2'] = predictionDF2['predictions'].values
<ipython-input-1-524c8d5248e4>:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,predictions2,predictions
4488,6.7,0.480,0.49,2.90,0.030,28.0,122.0,0.98926,3.13,0.40,13.000000,good,good,good
3862,6.5,0.270,0.19,6.60,0.045,98.0,175.0,0.99364,3.16,0.34,10.100000,good,good,good
2109,7.7,0.230,0.31,10.70,0.038,59.0,186.0,0.99690,3.12,0.55,9.500000,good,good,good
4054,7.2,0.170,0.28,17.55,0.050,33.0,154.0,0.99971,2.94,0.43,9.000000,good,good,good
4515,7.1,0.280,0.26,2.80,0.039,50.0,118.0,0.99080,3.06,0.59,11.200000,good,good,good
4102,6.9,0.370,0.23,9.50,0.057,54.0,166.0,0.99568,3.23,0.42,10.000000,bad,bad,bad
4796,6.4,0.105,0.29,1.10,0.035,44.0,140.0,0.99142,3.17,0.55,10.700000,good,good,good
1787,7.2,0.260,0.30,2.10,0.033,50.0,158.0,0.99090,3.33,0.43,12.100000,good,good,good
2475,7.4,0.560,0.09,1.50,0.071,19.0,117.0,0.99496,3.22,0.53,9.800000,bad,bad,bad
3904,5.0,0.455,0.18,1.90,0.036,33.0,106.0,0.98746,3.21,0.83,14.000000,good,good,good


In [2]:
##Use this to check the amount of error in the softmax classification
from sklearn.metrics import accuracy_score, jaccard_score
print(accuracy_score(testy, predictions))


0.7755102040816326
